In [1]:
from __future__ import division
import csv
from datetime import datetime
from collections import Counter

In [2]:
!curl -O http://web.mta.info/developers/data/nyct/turnstile/turnstile_150627.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.0M    0 24.0M    0     0  1119k      0 --:--:--  0:00:22 --:--:-- 1371k


In [3]:
with open('turnstile_150627.txt') as f:    #open the link to the data
    reader = csv.reader(f)    #read in the data
    rows = [[cell.strip() for cell in row] for row in reader]    #loop over the rows and witin the rows to extract the data and remove any whitespace from the beginning and end

In [4]:
assert rows.pop(0) == ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME',
                       'DIVISION', 'DATE', 'TIME', 'DESC', 'ENTRIES',
                       'EXITS']    #check to make sure that we have the correct variable names

In [5]:
raw_readings = {}    #an empty dictionary
for row in rows:    #loop over the elements in the list rows
    raw_readings.setdefault(tuple(row[:4]), []).append(tuple(row[4:]))    #loop over the rows, add new keys to the dictionary when the row info is not already present, append to existing data when it is

In [6]:
len(raw_readings)
type(raw_readings)

dict

In [7]:
array_of_times = []
count = 0
for key,value in raw_readings.iteritems():
    #if count > 5:
    #    break
    #else:
    for the_tuple in value:
        array_of_times.append(the_tuple[3])
    #    count += 1

In [8]:
#print array_of_times
import pandas as pd
import numpy as np
import matplotlib as plt

array_of_times = pd.Series(array_of_times)

array_of_times.describe()

for x in array_of_times.head():
    print x


#array_of_times.plot.pie()

24*60*60

00:00:00
04:00:00
08:00:00
12:00:00
16:00:00


86400

In [9]:
#raw_readings.keys()    #the dictionary raw_readings solves challenge 1

In [10]:
datetime_cumulative = {turnstile: [(datetime.strptime(date + time,'%m/%d/%Y%X'),int(in_cumulative))
                                   for _, _, date, time,_, in_cumulative, _ in rows]
                       for turnstile, rows in raw_readings.items()}    #make a new dictionary; the keys will be the turnstiles, and each will have a list of tuples consisting of date and cumulative total

In [11]:
for rows in datetime_cumulative.values():    #loop over the time series
    assert rows == sorted(rows)    #and ensure that each is in the correct order in time - note that tuples are sorted by first element

In [12]:
datetime_count_times = {turnstile: [[rows[i][0],
                                     rows[i+1][1] - rows[i][1],
                                     rows[i+1][0] - rows[i][0]]
                                    for i in range(len(rows) - 1)]
                        for turnstile, rows in datetime_cumulative.items()}    #make a new dictionary; keys will be the turnstiles, each will have a list of lists, each of which will give date, change in entries from last time, and elasped time

In [13]:
datetime_counts = {turnstile: [(time, count)
                               for (time, count, _) in rows
                               if 0 <= count <= 5000]
                   for turnstile, rows in datetime_count_times.items()}    #make one more dictionary; keys will be turnstiles, each will have a list of lists with date and change in entries

In [14]:
#datetime_counts.keys()    #the dictionary datetime_counts solves challenge 2

In [15]:
day_counts = {}    #an empty dictionary to hold the total counts
for turnstile, rows in datetime_counts.items():    #loop over key/data pairs in datetime_counts
    by_day = {}    #for each make a new empty dictionary
    for time, count in rows:    #loop over the time and the count
        day = time.date()    #keep only the day info
        by_day[day] = by_day.get(day, 0) + count    #pull out the data associated with key <day> and add the count to it - if <day> is in the dictionary return 0
    day_counts[turnstile] = sorted(by_day.items())    #save the count by turnstile in the dictionary

In [16]:
day_counts.items()    #the dictionary day_counts solves challenge 3

[(('A030', 'R083', '01-06-00', '23 ST-5 AVE'),
  [(datetime.date(2015, 6, 20), 1008),
   (datetime.date(2015, 6, 21), 767),
   (datetime.date(2015, 6, 22), 2438),
   (datetime.date(2015, 6, 23), 2651),
   (datetime.date(2015, 6, 24), 2680),
   (datetime.date(2015, 6, 25), 2683),
   (datetime.date(2015, 6, 26), 1951)]),
 (('N134', 'R385', '00-00-00', 'ROCKAWAY BLVD'),
  [(datetime.date(2015, 6, 20), 2),
   (datetime.date(2015, 6, 21), 3),
   (datetime.date(2015, 6, 22), 9),
   (datetime.date(2015, 6, 23), 5),
   (datetime.date(2015, 6, 24), 11),
   (datetime.date(2015, 6, 25), 9),
   (datetime.date(2015, 6, 26), 7)]),
 (('B014', 'R148', '00-00-01', 'PARKSIDE AVE'),
  [(datetime.date(2015, 6, 20), 1489),
   (datetime.date(2015, 6, 21), 1356),
   (datetime.date(2015, 6, 22), 2073),
   (datetime.date(2015, 6, 23), 2106),
   (datetime.date(2015, 6, 24), 2180),
   (datetime.date(2015, 6, 25), 2131),
   (datetime.date(2015, 6, 26), 2044)]),
 (('A013', 'R081', '01-03-00', '49 ST-7 AVE'),
  [(d